In [1]:
cd ../../../cd_hit_py/

/Users/in-divye.singh/Documents/Projects/cd_hit_py


In [2]:
from cd_hit import CD_HIT

In [3]:
cd ../MIC_predictor/src/

/Users/in-divye.singh/Documents/Projects/MIC_predictor/src


In [4]:
import biovec
import numpy as np
import pandas as pd
from itertools import chain, combinations
from collections import Counter

from utils import *

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [5]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [6]:
def pearson_score(y_true, y_pred):
    pcc = pearsonr(y_true, y_pred)
    return pcc[0]

pcc_scorer = make_scorer(pearson_score)

In [7]:
avp_ic50 = pd.read_csv("../data/raw/AVP-IC50Pred_train.csv")
ha_avp = pd.read_csv("../data/raw/HA_AVP.csv")

In [8]:
df = pd.concat([avp_ic50[['Sequence','MIC']], ha_avp], axis=0).drop_duplicates(['Sequence']).reset_index(drop=True)
df = sequence_filtering(df)

In [9]:
df['pIC50'] = df['MIC'].apply(lambda x: -np.log(x*1e-6))

In [10]:
family = pd.read_csv("../data/raw/712pep_family.csv")

In [11]:
cov_seq = family[(family['Abbreviation'] == 'SARS-CoV')].reset_index(drop=True)

In [12]:
cov_seq

,Sequence,Abbreviation,Family
0,SNNTIAIPTNFSISITTEVM,SARS-CoV,Coronaviridae
1,FKLPLGINITNFRAILTAFS,SARS-CoV,Coronaviridae
2,VLYNSTFFSTFKCYGVSATK,SARS-CoV,Coronaviridae
3,GIGVTQNVLYENQKQIANQF,SARS-CoV,Coronaviridae
4,PALNCYWPLNDYGFYTTSGI,SARS-CoV,Coronaviridae
5,FGGASCCLYCRCHIDHPNPKGFCDLKGKY,SARS-CoV,Coronaviridae
6,GGASCCLYCRCH,SARS-CoV,Coronaviridae
7,IEEINKKVEEIQKKIEELNKKAEELNKKLEELQKK,SARS-CoV,Coronaviridae
8,YQDVNCTDVSTAIHADQLTP,SARS-CoV,Coronaviridae
9,PTTFMLKYDENGTITDAVDC,SARS-CoV,Coronaviridae


In [13]:
df_cov = df.merge(cov_seq,how='right',on='Sequence').reset_index(drop=True)

In [14]:
df_cov#.to_csv("../data/raw/hiv_cdhit_filtered.csv", index=False)

,Sequence,MIC,pIC50,Abbreviation,Family
0,SNNTIAIPTNFSISITTEVM,278.801,8.185012,SARS-CoV,Coronaviridae
1,FKLPLGINITNFRAILTAFS,266.434,8.230384,SARS-CoV,Coronaviridae
2,VLYNSTFFSTFKCYGVSATK,262.667,8.244623,SARS-CoV,Coronaviridae
3,GIGVTQNVLYENQKQIANQF,262.539,8.245111,SARS-CoV,Coronaviridae
4,PALNCYWPLNDYGFYTTSGI,258.330,8.261273,SARS-CoV,Coronaviridae
5,FGGASCCLYCRCHIDHPNPKGFCDLKGKY,160.000,8.740337,SARS-CoV,Coronaviridae
6,GGASCCLYCRCH,160.000,8.740337,SARS-CoV,Coronaviridae
7,IEEINKKVEEIQKKIEELNKKAEELNKKLEELQKK,100.000,9.210340,SARS-CoV,Coronaviridae
8,YQDVNCTDVSTAIHADQLTP,61.704,9.693162,SARS-CoV,Coronaviridae
9,PTTFMLKYDENGTITDAVDC,60.018,9.720866,SARS-CoV,Coronaviridae


In [15]:
def get_physicochemical_properties(df):
    params = ['aromaticity', 'helix', 'turn', 'sheet', 'gravy', 'net_charge_at_pH7point4']

    prop = []
    for seq in df.Sequence:
        X = ProteinAnalysis(seq)
        aromaticity = X.aromaticity()
        sec_struc = X.secondary_structure_fraction()
        helix = sec_struc[0]
        turn = sec_struc[1]
        sheet = sec_struc[2]
        gravy = X.gravy() # hydrophobicity related
        net_charge_at_pH7point4 = X.charge_at_pH(7.4)

        prop.append([aromaticity, helix, turn, sheet, gravy, net_charge_at_pH7point4])
    return pd.DataFrame(prop, columns=params)

In [16]:
aa_freq = reduce_by_kmer_frequency(df_cov)

In [17]:
uniprot_embedding = biovec.models.load_protvec("../data/embeddings/uniprot__kmer_3_contextWindow_10_vector_100_reduction_None")

avg_protvec = convert_sequences_to_avg_vectors(df_cov['Sequence'], uniprot_embedding, kmer=3)
avg_protvec = avg_protvec.reset_index(drop=True)

/Users/in-divye.singh/anaconda3/envs/life-science/lib/python3.6/site-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Creating vectors: 100%|██████████| 27/27 [00:00<00:00, 983.24sequence/s]


In [18]:
physicochemical_prop = get_physicochemical_properties(df_cov)

In [19]:
X = pd.concat([aa_freq, avg_protvec, physicochemical_prop[['helix','turn','sheet']]], axis=1)

In [20]:
y = df_cov[['pIC50', 'MIC']]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
y_train_pmic, y_train_mic = y_train['pIC50'], y_train['MIC']
y_test_pmic, y_test_mic = y_test['pIC50'], y_test['MIC']

In [23]:
from scipy.stats import pearsonr

In [24]:
from sklearn.model_selection import cross_val_score, GridSearchCV, LeaveOneOut

In [25]:
def multi_objective_score(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred)
    pcc = pearson_score(y_true, y_pred)
    return mape - 10*pcc
multi_objective_scorer = make_scorer(multi_objective_score, greater_is_better=False)

In [30]:
max_n_neighbors = int(np.sqrt(X_train.shape[0]))
param_grid = {
    'n_neighbors': range(1, max_n_neighbors),
    'weights': ['uniform', 'distance'],
    'metric': ["euclidean", "manhattan", "chebyshev"]
}
knn = KNeighborsRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = knn, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2, scoring=mape_scorer)

In [31]:
grid_search.fit(X_train, y_train_pmic)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    2.0s finished


GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan', 'chebyshev'],
                         'n_neighbors': range(1, 4),
                         'weights': ['uniform', 'distance']},
             scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False),
             verbose=2)

In [32]:
grid_search.best_params_

{'metric': 'chebyshev', 'n_neighbors': 3, 'weights': 'uniform'}

In [33]:
best_grid = grid_search.best_estimator_

In [34]:
y_pred_pmic = best_grid.predict(X_test)

In [35]:
mean_absolute_percentage_error(y_test_pmic, y_pred_pmic)

12.358114606753727

In [40]:
y_pred_mic = np.exp(-y_pred_pmic)/1e-6

In [41]:
ape_mic = 100*np.abs(y_test_mic-y_pred_mic)/y_test_mic

In [42]:
mean_absolute_percentage_error(y_test_mic, y_pred_mic)

382.43884589881236

In [45]:
pearson_score(y_test_mic, y_pred_mic)

0.8541007832661847

In [46]:
list(zip(y_test_mic.round(4), y_pred_mic.round(4), ape_mic))

[(61.704, 7.8464, 87.28382241738854),
 (11.0, 18.8052, 70.9564103442852),
 (60.018, 19.3792, 67.71094175269),
 (0.14, 2.8582, 1941.5379125206996),
 (278.801, 98.4268, 64.69640787478508),
 (33.889, 55.0519, 62.44758048302582)]

In [47]:
loo = LeaveOneOut()

from tqdm import tqdm

result_df = pd.DataFrame(columns = list(df_cov.columns)+["y_pred_pmic", "y_pred_mic", "ape_pmic", "ape_mic"])
for train_index, test_index in tqdm(loo.split(X)):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    y_train_pmic, y_train_mic = y_train['pIC50'], y_train['MIC']
    y_test_pmic, y_test_mic = y_test['pIC50'], y_test['MIC']
    knn = KNeighborsRegressor(**grid_search.best_params_)
    _ = knn.fit(X_train, y_train_pmic)
    y_pred_pmic = knn.predict(X_test)
    y_pred_mic = np.exp(-y_pred_pmic)/1e-6
    ape = 100*np.abs(y_test_pmic-y_pred_pmic)/y_test_pmic
    ape_mic = 100*np.abs(y_test_mic-y_pred_mic)/y_test_mic
    df_val = df_cov.iloc[test_index,:].values[0].tolist()
    res = np.append(df_val, [y_pred_pmic[0], y_pred_mic[0], ape.values[0], ape_mic.values[0]])
    res = pd.DataFrame([res], columns = list(df_cov.columns)+["y_pred_pmic", "y_pred_mic", "ape_pmic", "ape_mic"])
    result_df = result_df.append(res)
result_df = result_df[["Sequence", "pIC50", "y_pred_pmic", "ape_pmic", "MIC", "y_pred_mic", "ape_mic"]]

27it [00:00, 97.77it/s]


In [48]:
result_df#.to_csv("../results/SVM_HIV_CoV_pMIC_to_MIC_rbf_c_100_gamma_2.csv", index=False)

,Sequence,pIC50,y_pred_pmic,ape_pmic,MIC,y_pred_mic,ape_mic
0,SNNTIAIPTNFSISITTEVM,8.185012292283764,10.757760044913107,31.432423810221316,278.801,21.279635020863918,92.3674466659503
0,FKLPLGINITNFRAILTAFS,8.230383999814125,10.099898500174003,22.714790712099212,266.434,41.08372500462046,84.58014930353465
0,VLYNSTFFSTFKCYGVSATK,8.244623487739776,10.490348829563159,27.238664629899755,262.66700000000003,27.80349489384871,89.41492654431325
0,GIGVTQNVLYENQKQIANQF,8.245110915539666,13.47017920598782,63.37171620821258,262.539,1.4124578628409397,99.46200074547365
0,PALNCYWPLNDYGFYTTSGI,8.26127272058809,9.656496868684387,16.888731255890264,258.33,64.00835932477052,75.2222508710678
0,FGGASCCLYCRCHIDHPNPKGFCDLKGKY,8.740336742730447,8.733019335886814,0.0837199647910487,160.0,161.17507911772856,0.7344244485803486
0,GGASCCLYCRCH,8.740336742730447,8.415410983686103,3.7175427973595285,160.0,221.42846368592592,38.3927898037037
0,IEEINKKVEEIQKKIEELNKKAEELNKKLEELQKK,9.210340371976184,14.538661818094562,57.851515046399165,100.0,0.48522078727788287,99.51477921272212
0,YQDVNCTDVSTAIHADQLTP,9.693161799332577,11.449926748119843,18.123755541852486,61.70399999999999,10.650254726434039,82.73976609873908
0,PTTFMLKYDENGTITDAVDC,9.720866040733176,9.708240615145405,0.12987963762556784,60.018,60.78055646351822,1.270546275314432


In [49]:
result_df['ape_pmic'].astype('float').mean()

20.507920844613395

In [50]:
result_df['ape_mic'].astype('float').mean()

1206.2696592008815

In [51]:
pearsonr(result_df['MIC'].astype('float'), result_df['y_pred_mic'].astype('float'))

(0.27629951704541045, 0.16298823325233647)